In [180]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [181]:
dists = pd.read_csv("data/atlas_open_data_files/distributions.csv")
dist_changes = pd.read_csv("data/atlas_open_data_files/distribution_changes.csv")
grid_coords = pd.read_csv("data/atlas_open_data_files/grid_square_coordinates_lookup.csv")
pct_bench_species = pd.read_csv("data/atlas_open_data_files/percent_benchmark_species_detected.csv")
species_lookup = pd.read_csv("data/atlas_open_data_files/species_lookup_nocase.csv", encoding="ISO-8859-1")
redlist = pd.read_csv("data/conservation/redlist_2021.csv")
amberlist = pd.read_csv("data/conservation/amberlist_2021.csv")

#### Misc stuff

In [182]:
# The number of atlases in the dataset
dists["period"].unique()

array(['1968-72', '2008-11', '2007/08-10/11', '1980/81-1982/83',
       '1988-91'], dtype=object)

In [183]:
# These are the top 10 birds which were spotted most in the Big Garden 
# Birdwatch (2023) https://www.rspb.org.uk/whats-happening/big-garden-birdwatch
HOUSE_SPARROW = 459
BLUE_TIT = 436
STARLING = 457
WOODPIDGEON = 270
BLACKBIRD = 371
ROBIN = 345
GOLDFINCH = 471
GREAT_TIT = 437
MAGPIE = 450
LONG_TAILED_TIT = 431

# Ideas
- Regression of distribution changes of common garden birds?
- Classifying birds that need special concern?
- To what extent does the presence/movement of passerines affect the movement of birds of prey?
    - Regression where y = bird of prey and x = passerine

- **Feed data in to NN which predicts whether a bird will be on the latest Red List or not**

# Predicting conservation status using citizen science bird atlases

In [184]:
amberlist

,english_name,prev_status
0,quail,nc
1,stone-curlew,nc
2,tawny owl,nc
3,whooper swan,nc
4,oystercatcher,nc
...,...,...
98,reed bunting,nc
99,great white egret,na
100,guillemot,nc
101,gannet,nc


In [185]:
amberlist_merged = amberlist.merge(species_lookup, on="english_name", how="left")
amberlist_merged

,english_name,prev_status,speccode,scientific_name,taxonomic_rank
0,quail,nc,132.0,coturnix coturnix,species
1,stone-curlew,nc,155.0,burhinus oedicnemus,species
2,tawny owl,nc,285.0,strix aluco,species
3,whooper swan,nc,48.0,cygnus cygnus,species
4,oystercatcher,nc,152.0,haematopus ostralegus,species
...,...,...,...,...,...
98,reed bunting,nc,524.0,emberiza schoeniclus,species
99,great white egret,na,38.0,ardea alba,species
100,guillemot,nc,260.0,uria aalge,species
101,gannet,nc,25.0,morus bassanus,species


In [186]:
new_amberlist = amberlist_merged[["speccode", "english_name", "prev_status"]]
new_amberlist.to_csv("data/conservation/new_amberlist_2021.csv", index=False)